In [1]:
from readers import *
from models import *
from datalib import *
import config.config_create_trajectory as config
import matplotlib.pyplot as plt
import time
%matplotlib notebook

Failed to load Python extension for LZ4 support. LZ4 compression will not be available.


In [2]:
print("Reading demonstration data")

position_datasets = []
orientation_datasets = []
velocity_datasets = []
rotational_velocity_datasets = []

for demo in config.demos:
    print(f'Reading demonstration file {demo}')
    franka_reader = FrankaStateReader(demo)

    position_dataset = PositionDataSet()
    orientation_dataset = PositionDataSet()
    velocity_dataset = PositionDataSet()
    rotational_velocity_dataset = PositionDataSet()

    for i in range(len(franka_reader.msgs)):
        dp = franka_reader.next_datapoint()
        time = dp.time
        dp = dp.value

        position_dataset.append(PositionDataPoint(time, dp.position))
        orientation_dataset.append(PositionDataPoint(time, dp.euler_angles))
        velocity_dataset.append(PositionDataPoint(time, dp.velocity))
        rotational_velocity_dataset.append(PositionDataPoint(time, dp.rotational_velocity))

    position_dataset.align_time()
    orientation_dataset.align_time()
    velocity_dataset.align_time()
    rotational_velocity_dataset.align_time()

    position_datasets.append(position_dataset)
    orientation_datasets.append(orientation_dataset)
    velocity_datasets.append(velocity_dataset)
    rotational_velocity_datasets.append(rotational_velocity_dataset)

impact_intervals = []
for delayed_demo_impact_intervals in config.impact_intervals:
    demo_impact_intervals = []
    for delayed_impact_interval in delayed_demo_impact_intervals:
        impact_interval = ()
        for i in range(len(delayed_impact_interval)):
            if i == 0:
                impact_interval += (delayed_impact_interval[0] - config.impact_detection_delay[0],)
            else:
                impact_interval += (delayed_impact_interval[i] - config.impact_detection_delay[-1],)
        demo_impact_intervals.append(impact_interval)
    impact_intervals.append(demo_impact_intervals)

datasets_handle = RobotDataSets(position_datasets, velocity_datasets, orientation_datasets, rotational_velocity_datasets, impact_intervals, config.impact_duration)

print("Done")

Reading demonstration data
Reading demonstration file data/demo1.bag
Reading demonstration file data/demo4.bag
Reading demonstration file data/demo5.bag
Reading demonstration file data/demo10.bag
Reading demonstration file data/demo11.bag
Reading demonstration file data/demo12.bag
Reading demonstration file data/demo13.bag
Reading demonstration file data/demo14.bag
Reading demonstration file data/demo15.bag
Reading demonstration file data/demo19.bag
Reading demonstration file data/demo21.bag
Done


In [3]:
print("Filtering and extending demonstration data")

datasets_handle.filter_position_data(config.position_filter)
datasets_handle.filter_velocity_data(config.velocity_filter)
datasets_handle.filter_orientation_data(config.orientation_filter)
datasets_handle.extend_position_data(config.position_extender)
datasets_handle.extend_orientation_data(config.orientation_extender)

print("Done")

Filtering and extending demonstration data
Done


In [4]:
print("Creating ProMPs")

position_promps = datasets_handle.create_position_promps(config.rbf_width, config.n_rbfs_per_second)
orientation_promps = datasets_handle.create_orientation_promps(config.rbf_width, config.n_rbfs_per_second)

print("Done")

Creating ProMPs
-1.6220173835754395 0.0 -1.6199891567230225 0.20000000000000018
[Time: -1.6219596862792969, Value: [0.4286549572537681, 0.0007768235698326605], Time: 0.20199999999997775, Value: [0.3655690226911169, -0.2028625816389394]]
[Time: -1.6219415664672852, Value: [0.43350683355058006, -0.00039079276221305005], Time: 0.20199999999997775, Value: [0.3602698424034377, -0.23943324356266704]]
[Time: -1.6209757328033447, Value: [0.4276634872659846, -3.341595410275551e-05], Time: 0.20199999999997775, Value: [0.35370346875168684, -0.25406026700735423]]
[Time: -1.6199891567230225, Value: [0.450523639688851, 1.2048381996492046e-05], Time: 0.20199999999997775, Value: [0.43284716464095, -0.1009468617215604]]
[Time: -1.6209945678710938, Value: [0.4437769553571866, 0.0004088529024664843], Time: 0.20199999999997775, Value: [0.4291081650534939, -0.07459161515261387]]
[Time: -1.6219422817230225, Value: [0.43775859987481364, -0.00018138546616699379], Time: 0.20199999999997775, Value: [0.426928720

[Time: -0.20000000000000018, Value: [-0.02888286347833556, 0.6978022457243477], Time: 3.4668631553649902, Value: [-0.0563219500203159, 0.002154653489615861]]
[Time: -0.20000000000000018, Value: [-0.04550514015641245, 0.7825495258711956], Time: 3.467919111251831, Value: [-0.08786893882123303, -0.0011576318485747024]]
[Time: -0.20000000000000018, Value: [-0.06762612709310399, 0.9038226435760526], Time: 3.4659695625305176, Value: [-0.06349721835712996, 9.983562952434235e-05]]
[Time: -0.20000000000000018, Value: [-0.05780498709750813, 0.8819743779725523], Time: 3.4669198989868164, Value: [-0.09259709507742303, -0.00022778887877428683]]
[Time: -0.20000000000000018, Value: [-0.04356987479873939, 0.7220537472260337], Time: 3.465895414352417, Value: [-0.07800762351310564, -0.0011215928208055661]]
[Time: -0.19999999999999996, Value: [-0.023437285012690823, 0.6871292880212032], Time: 3.466968536376953, Value: [-0.040014671682726835, -0.0010420252276953778]]
[Time: -0.20000000000000018, Value: [-

In [5]:
if config.write_mps:
    print("Saving ProMPs to file")

    json_object = dict()
    json_object['rotation_matrix'] = datasets_handle.rotation_matrix.tolist()
    json_object['phases'] = []
    for phase in range(datasets_handle.n_phases):
        pos_promp_dicts = []
        or_promp_dicts = []
        for i in range(3):
            pos_promp_dicts.append(position_promps[phase].value[i].to_dict())
            or_promp_dicts.append(orientation_promps[phase].value[i].to_dict())
        phase_promps = dict()
        phase_promps['position_promps'] = pos_promp_dicts
        phase_promps['orientation_promps'] = or_promp_dicts
        json_object['phases'].append(phase_promps)

    with open(config.output_file, 'w', encoding='utf-8') as f:
        json.dump(json_object, f, ensure_ascii=False, indent=4)

Saving ProMPs to file
